### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [ ]:
!pip install evaluate
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 903.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.8 MB/s eta 0:00:00


In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.3 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.13.1-py3-none-any.whl size=1350298 sha256=0949241e76df1a423f0756815f8d797dfb0982a77fba5bb5198b8f53c88246bf
  Stored in directory: /root/.cache/pip/wheels/0f/fb/b5/b159b3500525eca167d8ca6e3a7e224b6075045cac90f47cf7
Successfully built deepspeed
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


### Load data and model

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [ ]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
roc_auc_score = evaluate.load("roc_auc")

In [ ]:
from tqdm import tqdm
import numpy as np


class Evaluation:
    def __init__(self, model, data, device, workers, batch_size):
        self.model = model
        self.loader = torch.utils.data.DataLoader(
            data, batch_size=batch_size, num_workers=workers, shuffle=False, collate_fn=transformers.default_data_collator
        )
        self.device = device
        self.pred = []
        self.labels = []
        self.pred_scores = []

    def evaluate(self):
        self.model = self.model.to(device)

        for batch in tqdm(self.loader):
            with torch.autocast(device_type=device, dtype=torch.float16):
                with torch.no_grad():
                    predicted = self.model(
                        input_ids=batch['input_ids'].to(device),
                        attention_mask=batch['attention_mask'].to(device),
                        token_type_ids=batch['token_type_ids'].to(device)
                    )
                    pred_scores = torch.softmax(predicted.logits, dim=1).detach().cpu().numpy()
                    pred = pred_scores.argmax(axis=1)
                    self.pred_scores.append(pred_scores)
                    self.pred.append(pred)
                    self.labels.append(batch["labels"].cpu().numpy())
        self.pred_scores = np.concatenate(self.pred_scores)
        self.pred = np.concatenate(self.pred)
        self.labels = np.concatenate(self.labels)

    def accuracy(self):
        return accuracy.compute(predictions=self.pred, references=self.labels)

    def precision(self):
        return precision.compute(predictions=self.pred, references=self.labels)

    def recall(self):
        return recall.compute(predictions=self.pred, references=self.labels)

    def roc_auc(self):
        return roc_auc_score.compute(prediction_scores=self.pred_scores[:, 1], references=self.labels)

    def calc_metric(self):
        return {**self.accuracy(), **self.precision(), **self.recall(), **self.roc_auc()}

In [ ]:
BATCH_SIZE = 32
device = "cuda" if torch.cuda.is_available() else "cpu"

eval = Evaluation(model, val_set, device, workers=2, batch_size=BATCH_SIZE)
eval.evaluate()
metrics = eval.calc_metric()
accuracy_metric = metrics["accuracy"]
metrics

100%|██████████| 1264/1264 [01:26<00:00, 14.68it/s]


{'accuracy': 0.9083601286173634,
 'precision': 0.8684904416611734,
 'recall': 0.8851192475646624,
 'roc_auc': 0.9661216978107027}

In [ ]:
assert 0.9 < accuracy_metric < 0.91

### Task 2: train the model (5 points)

For this task, you have two options:

[+] __Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [ ]:
model_name = "microsoft/deberta-v3-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(eval_pred):
    predictions_scores, labels = eval_pred
    prediction_scores = torch.softmax(torch.tensor(predictions_scores), dim=1).detach().numpy()
    predictions = np.argmax(prediction_scores, axis=1)
    metrics = accuracy.compute(predictions=predictions, references=labels) | \
        precision.compute(predictions=predictions, references=labels) | \
        recall.compute(predictions=predictions, references=labels) | \
        roc_auc_score.compute(prediction_scores=prediction_scores[:, 1], references=labels)
    return metrics

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

training_args = TrainingArguments(
    output_dir="debert-qqp",
    learning_rate=2e-5,
    per_device_train_batch_size=96,
    per_device_eval_batch_size=96,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=qqp_preprocessed["train"],
    eval_dataset=qqp_preprocessed["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Roc Auc
1,0.228600,0.220569,0.910735,0.850317,0.919382,0.969926
2,0.170500,0.208466,0.916077,0.867110,0.911790,0.973137


TrainOutput(global_step=7582, training_loss=0.21627569148295853, metrics={'train_runtime': 7391.8613, 'train_samples_per_second': 98.445, 'train_steps_per_second': 1.026, 'total_flos': 4.78668109446697e+16, 'train_loss': 0.21627569148295853, 'epoch': 2.0})

In [ ]:
eval = Evaluation(model, qqp_preprocessed['validation'], device, workers=2, batch_size=BATCH_SIZE)
eval.evaluate()
metrics = eval.calc_metric()
metrics

100%|██████████| 1264/1264 [02:09<00:00,  9.78it/s]


{'accuracy': 0.9160277022013357,
 'precision': 0.8670457449527217,
 'recall': 0.9117232112865301,
 'roc_auc': 0.9731358448305936}

### Task 3: try the full pipeline (2 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [ ]:
from datasets import Dataset


question = "How is the life of a math student? Could you describe your own experiences?"

def preprocess_function_text(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    return result

def get_texts(data):
    flatten = []
    for d in data:
        flatten.append(d["text1"])
        flatten.append(d["text2"])
    return set(flatten)

def find_duplicates(question, model, batch_size=32):
    data = qqp["train"]
    data = get_texts(data)
    if question in data:
        data.remove(question)
    data = Dataset.from_list([{"text1": question, "text2": question_from_dataset} for question_from_dataset in data])
    data = data.map(preprocess_function_text, batched=True)
    dataloader = torch.utils.data.DataLoader(
        data, batch_size=batch_size, num_workers=2, shuffle=False, collate_fn=transformers.default_data_collator
    )
    pred_scores = []

    for batch in tqdm(dataloader):
        with torch.autocast(device_type=device, dtype=torch.float16):
            with torch.no_grad():
                predicted = model(
                    input_ids=batch['input_ids'].to(device),
                    attention_mask=batch['attention_mask'].to(device),
                    token_type_ids=batch['token_type_ids'].to(device)
                )
                pred_score = torch.softmax(predicted.logits, dim=1).detach().cpu().numpy()
                pred_scores.append(pred_score)
    pred_scores = np.concatenate(pred_scores)
    return data, pred_scores

data, scores = find_duplicates(question, model)


Map:   0%|          | 0/493873 [00:00<?, ? examples/s]

100%|██████████| 15434/15434 [26:10<00:00,  9.83it/s]


In [ ]:
def get_top_k_q(data, scores, k=5):
    ind = np.argpartition(scores[:, 1], -k)[-k:][::-1]
    result = [{"score": scores[i, 1], "text": data[int(i)]["text2"]} for i in ind]
    return result

get_top_k_q(data, scores)

[{'score': 0.67955095,
  'text': 'What is height og SSC CGL preparation what is height of SSC CGL preparation?'},
 {'score': 0.44825834,
  'text': 'How can I create an app similar to Uber/Ola on a small scale? How can I create an app similar to Uber/Ola on a small scale?'},
 {'score': 0.32815254,
  'text': 'Are Chinese characters inefficient? Quora Top Stories from Your Feed Your Quora Digest Are Chinese characters inefficient'},
 {'score': 0.32680833,
  'text': 'How many hours a day do Harvard students sleep? At what time do Harvard students sleep?'},
 {'score': 0.2942657,
  'text': 'Is it possible to make a ghost appear at school in front of you? How can you make a ghost appear in front of you during class or in school?'}]

__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.